In [17]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
df=pd.read_csv('df_1000.csv')

In [ ]:
df.shape

In [ ]:
for bin_feature in [ 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
    df[bin_feature], uniques = pd.factorize(df[bin_feature])

In [ ]:
set(df['FLAG_OWN_CAR'])

In [ ]:
set(df['FLAG_OWN_REALTY'])

In [ ]:
df.shape

In [ ]:
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [ ]:
df, cat_cols = one_hot_encoder(df, nan_as_category=False)

In [ ]:
df.shape

In [12]:
df= df.drop(USELESS_COLUMNS,axis=1)

In [13]:
df.shape

(356255, 1129)

In [14]:
import xgboost as xgb
from xgboost import plot_importance

/Users/shuozhang/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [15]:
train_df = df[df['TARGET'].notnull()]
test_df = df[df['TARGET'].isnull()]
print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))

Starting LightGBM. Train shape: (307511, 1129), test shape: (48744, 1129)


In [ ]:
folds = KFold(n_splits= 5, shuffle=True, random_state=1001)
    # Create arrays and dataframes to store results
oof_preds = np.zeros(train_df.shape[0])
sub_preds = np.zeros(test_df.shape[0])
feature_importance_df = pd.DataFrame()
feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
dtest=xgb.DMatrix(test_df[feats])
    
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
    dtrain = xgb.DMatrix(train_df[feats].iloc[train_idx],train_df['TARGET'].iloc[train_idx])
    dvalid = xgb.DMatrix(train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx])
    valid_y=train_df['TARGET'].iloc[valid_idx]

       # xgb
    params = {'eval_metric': 'auc',
              'objective': 'binary:logistic',
              'booster':'gbtree',
              'tree_method': 'hist',
              'nthread' : 4,
              'eta' : 0.001,
               'max_leaves': 40,
              'max_depth' : 16,
              'max_bin': 255,
              'min_child_weight' : 4,
              'subsample' : 0.5,
              'colsample_bytree' : 0.5,
              'colsample_bylevel' : 1,
              'alpha' : 0.001,
              'lambda' : 0.001,
              'scale_pos_weight': 1}
    watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
        
    model=xgb.train(params, dtrain, 10000, watchlist, maximize=True, early_stopping_rounds = 100, verbose_eval=100)
    oof_preds[valid_idx] = model.predict(dvalid, ntree_limit=model.best_ntree_limit)
    sub_preds += model.predict(dtest,ntree_limit=model.best_ntree_limit) / folds.n_splits

    fold_importance_df = pd.DataFrame()
    fold_importance_df = pd.DataFrame(model.get_fscore().items(), columns=['feature','importance']).sort_values('importance', ascending=False)
    fold_importance_df["fold"] = n_fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
    del model, dtrain, dvalid, valid_y

print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    # Write submission file and plot feature importance
test_df['TARGET'] = sub_preds
test_df[['SK_ID_CURR', 'TARGET']].to_csv('xgb_1000feature.csv', index= False)

In [ ]:
cols = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]
plt.figure(figsize=(8, 10))
sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
plt.title('XGB Features (avg over folds)')
plt.tight_layout
plt.show()

In [ ]:
feature_importance_df.to_csv('feature_importance_xgb1000features.csv', index=False)